In [12]:
import numpy as np
from tensorflow.keras.utils import to_categorical

raw_text = '''
I get on with life as a programmer,
I like to contemplate beer.
But when I start to daydream,
My mind turns straight to wine.

Do I love wine more than beer?

I like to use words about beer.
But when I stop my talking,
My mind turns straight to wine.

I hate bugs and errors.
But I just think back to wine,
And I'm happy once again.

I like to hang out with programming and deep learning.
But when left alone,
My mind turns straight to wine.
'''

tokens = raw_text.split()
raw_text =' '.join(tokens)
char_vocab = sorted(list(set(raw_text)))
vocab_size = len(char_vocab)

char_to_index = dict((char, index) for char,index in enumerate(char_vocab) )
print(char_to_index)

{0: ' ', 1: "'", 2: ',', 3: '.', 4: '?', 5: 'A', 6: 'B', 7: 'D', 8: 'I', 9: 'M', 10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f', 16: 'g', 17: 'h', 18: 'i', 19: 'j', 20: 'k', 21: 'l', 22: 'm', 23: 'n', 24: 'o', 25: 'p', 26: 'r', 27: 's', 28: 't', 29: 'u', 30: 'v', 31: 'w', 32: 'y'}
